# 프로젝트 : 멋진 작사가 만들기 


## 데이터 불러오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 데이터 정제 

In [4]:
import os, re 
import numpy as np
import tensorflow as tf

file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [8]:

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   
    if sentence[-1] == ":": continue  

    if idx > 15 : break   
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?
Resolved. resolved.
First, you know Caius Marcius is chief enemy to the people.


##  토큰화
다음과 같은 순서로 문장을 일정한 기준으로 쪼개는 토큰화 (Tokenize) 를 진행한다.

1. 소문자로 바꾸고, 양쪽 공백을 지움
2. 특수문자 양쪽에 공백을 넣음
3. 여러개의 공백은 하나의 공백으로 바꿈
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈
5. 다시 양쪽 공백을 지움
6. 문장 시작에는 <start>, 끝에는 <end>를 추가

In [9]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [25]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [12]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=12000,   # 단어장의 크기는 12,000 이상으로 설정
    filters=' ',
    oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc20d239a90>


In [13]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [14]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 15: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of
11 : you
12 : my
13 : a
14 : that
15 : ?


In [16]:
src_input = tensor[:, :-1]  # 소스 문장 생성

tgt_input = tensor[:, 1:]    # 타겟 문장 생성

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [18]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((12000, 20), (12000, 20)), types: (tf.int32, tf.int32)>

In [27]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,       # 총 데이터의 20%를 평가 데이터셋으로 사용
                                                          shuffle=True, 
                                                          random_state=34)     # 결과를 일정하게 보여주기위해 지정

In [28]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (19212, 20)
Target Train: (19212, 20)


19212개의 학습 데이터를 확인할 수 있다. 

### 모델  학습

In [21]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

TextGenerator는 1개의 embedding layer, 2개의 rnn(LSTM) layer, 1개의 Dense layer로 구성된다. 

embedding_size = word vector의 차원 수, 단어가 추상적으로 표현되는 크기.
hidden_size =  LSTM 레이어의 hidden state의 차원수

In [31]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10) # 10 epoch. val_loss =2.2를 줄일 수 있도록

Epoch 1/10
601/601 [==============================] - 107s 139ms/step - loss: 2.7053
Epoch 2/10
601/601 [==============================] - 84s 140ms/step - loss: 2.3846
Epoch 3/10
601/601 [==============================] - 84s 140ms/step - loss: 2.2284
Epoch 4/10
601/601 [==============================] - 84s 140ms/step - loss: 2.0811
Epoch 5/10
601/601 [==============================] - 84s 140ms/step - loss: 1.9216
Epoch 6/10
601/601 [==============================] - 84s 140ms/step - loss: 1.7395
Epoch 7/10
601/601 [==============================] - 84s 140ms/step - loss: 1.5375
Epoch 8/10
601/601 [==============================] - 84s 140ms/step - loss: 1.3302
Epoch 9/10
601/601 [==============================] - 84s 140ms/step - loss: 1.1308
Epoch 10/10
601/601 [==============================] - 84s 140ms/step - loss: 0.9550


### Step 5. 인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

In [44]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

     
    while True:
        predict = model(test_tensor)  
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]    

         
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   

In [45]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [50]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=15)

'<start> i love thee not a jar o the clock behind <end> '

In [56]:
generate_text(model, tokenizer, init_sentence="<start> show", max_len=15)

'<start> show d mastership in floating fortune s blows , <end> '

In [58]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=15)

'<start> you have made worms meat of me i have it , <end> '

In [59]:
generate_text(model, tokenizer, init_sentence="<start> mouse", max_len=15)

'<start> mouse in their election . <end> '

In [62]:
generate_text(model, tokenizer, init_sentence="<start> water", max_len=15)

'<start> water to be <unk> with our <unk> , <end> '

In [63]:
generate_text(model, tokenizer, init_sentence="<start> deep learning", max_len=15)

'<start> deep learning , fair <unk> , of all eyes , <end> '

In [64]:
generate_text(model, tokenizer, init_sentence="<start> hey", max_len=20)

'<start> <unk> , and potpan ! <end> '

In [65]:
generate_text(model, tokenizer, init_sentence="<start> mom", max_len=20)

'<start> <unk> , and potpan ! <end> '

In [67]:
generate_text(model, tokenizer, init_sentence="<start> Siri", max_len=20)

'<start> <unk> , and potpan ! <end> '

# 회고

1. 포함되지 않는 단어는 <unk>로 표현된다고 하는데 생각보다 많이 나온다. 
2. 어려웠던 부분은 optimizer = tf.keras.optimizers.Adam() 와 loss = tf.keras.losses.SparseCategoricalCrossentropy를 사용하는 부분이었다. 이번 노드에서 보이지 않았고 수정해야하는 부분을 찾는 것이 쉽지 않았다. 
3. tokenize를 할 때 oov_token이 뭔지 몰랐다. oov= out of vocabulary
4. embedding_size와 hidden_size를 다양하게 도전해보고 싶었으나 학습에 시간이 너무 많이 걸려 할 수 없었다. 
5. 지금은 학습 데이터로 사용된 자료들이 영어로 된 노래였다. 하지만, 한글로 된 자료를 이용해서도 만들어 보고 싶다. 한국어는 어순이 달라도 의미가 크게 달라지지 않아, 이상하더라도 듣는 사람이 알아 들을 수 있다. 물론, 시적 허용으로 이해 될 수도 있다. 이런 점에서 한국의 시를 이용해서 만들어 보고 싶다.
    - 시와 노래 가사는 크게 다른 바가 없다고 생각한다. 하지만, 시에는 시인의 생활 환경이나, 그 사람의 역사에 대한 내용이 담기기 마련이다. 
    - 특히, 지금 갑자기 "역사, 사회의 배경이 동일한 시대의 작품을 학습시키면 그 시대에 살았던 시인들과 비슷한 작품을 만들어 낼 수 있을까?"라는 의문이 생겼다. 우리나라는 100여 년간 일제 강점기와 광복, 6.25전쟁, 민주화 운동 등 그리 길지 않은 시간에 많은 굴곡진 역사가 있었고, 그 시대적 상황의 변화에 따라 많은 작품이 형성되었다. 시어 속에 숨겨진 중의적인 단어의 의미나 상징적인 단어들이 딥러닝 학습을 통해서 제대로 구현될 수 있을 지 궁금하다. 

6. 화가들의 그림을 학습해서 그 화가의 스타일로 그림을 그리는 AI에 대한 기사를 본 적이 있다. 
    https://nownews.seoul.co.kr/news/newsView.php?id=20211012601012
    
   그렇다면 작가의 글을 학습해서 그 작가의 스타일로 작품을 생성하는 것도 가능하리라 짐작한다.
    
   최근 상영했던 SF영화 '듄(dune)'은 동명의 소설을 원작으로 하고 있는데, 6편의 소설을 집필하고 작가는 생을 마감하게 되었다. 다른 작가나 아들이 소설을 완결짓기 위해서 노력했으나 원작자에 못 미치는 수준이라 독자들의 아쉬움이 크다고 했다. 그렇다면, AI로 원작자의 스타일을 학습시킨다면 원작자의 아이디어와 가장 근접한 작품을 만들어 낼 수 있지 않을까하는 기
    
7. 생성된 모든 문장이 자연스러운 것은 아니다. 그리고 다른  init_sentesnce를 넣었지만 알 수 없는 이유로 "and potpan !"이 계속 나오기도 한다. you have made worms meat of me i have it.(넌 나를 나를 벌레 고기로 만들었다..)라는 이상한 가사가 만들어지기도 했다. 
    
'<start> deep learning , fair <unk> , of all eyes , <end> '
    -> 꿈보다 해몽이라고 했던가. 꽤나 철학적인 가사가 탄생했다.  init sentence로 'deep learning'을 넣었는데 이렇게 철학적인 노래 가사가 탄생했다. Deep learning, fair of all eyes. 딥러닝, 공평한 모두의 눈. 우연의 조합이지만 딥러닝에 대한 정확한 통찰을 보여주는 가사가 만들어졌다. 
    
    => 학습할 데이터의 양을 아주 많이 늘리면 어색하거나 이상한 내용의 문제는 점차 줄어들 것이다. 
    
8. 이번 '작사가 인공지능 만들기' 프로젝트를 통해서 AI를 응용할 수 있는 다양한 방법들을 생각할 수 있었다.
    특히 다른 프로젝트에 비해서 응용해보고 싶은 마음이 많이 들었다. 
    
9. 4번째 epoch부터 loss는 2.2 이하로 낮아졌다. 최종 10번째 epoch에서는 0.9550의 아주 낮은 loss를 보였다. 
    그런데 여기서 의문이 든다. 지금 loss가 낮아진 결과 문장의 완성도가 높아진 것이 지금 이 수준인가? 그렇다면 epoch만이라도... 계속 늘려보고 싶다. 다음 기회에 계속.